In [ ]:
%load_ext watermark


In [ ]:
from IPython.display import display
import pandas as pd
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp

from pylib.munge._calc_fixprobs_from_traits import calc_fixprobs_from_traits
from pylib.viz._size_fixation_heatmap import size_fixation_heatmap


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "cupy-denovo-spatial2d-traits"
teeplot_subdir


## Prep Data


In [ ]:
dfpd = pd.read_parquet("https://osf.io/ag4ur/download")
display(dfpd.describe()), display(dfpd.head()), display(dfpd.tail());


In [ ]:
(apn,) = dfpd["tilePopSize"].unique()  # agents per node
apn


In [ ]:
dfxs = calc_fixprobs_from_traits(dfpd)
display(dfxs.describe()), display(dfxs.head()), display(dfxs.tail());


## Size Fixation Curves


In [ ]:
for errorbar in "sd", "se", "ci", None:
    tp.tee(
        sns.relplot,
        data=dfxs,
        x="population size",
        y="fixation probability",
        hue="genotype",
        col="available beneficial mutations",
        style="genotype",
        col_wrap=3,
        hue_order=["normomutator", "hypermutator"],
        style_order=["normomutator", "hypermutator"],
        aspect=1.5,
        height=2,
        kind="line",
        markers=True,
        teeplot_outexclude="post",
        errorbar=errorbar,
        teeplot_postprocess=f"""
teed.set(xscale="log", ylim=(-0.02, 1.02))
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.02)
teed.set_titles(col_template="{{col_var}} = {{col_name:.0f}}")
sns.move_legend(
    teed,
    "lower center",
    bbox_to_anchor=(0.25, 0.97),
    frameon=False,
    ncol=2,
    title=None,
)
    """,
    )


In [ ]:
import matplotlib.colors
import matplotlib.pyplot as plt

cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    "",
    ["chocolate", "darkviolet", "royalblue"],
)

for errorbar in "sd", "se", "ci", None:
    tp.tee(
        sns.relplot,
        data=dfxs[dfxs["genotype"] == "hypermutator"],
        x="population size",
        y="fixation probability",
        hue="available beneficial mutations",
        # style="available beneficial mutations",
        aspect=1.5,
        height=3,
        kind="line",
        markers=True,
        teeplot_outexclude="post",
        errorbar=errorbar,
        palette=cmap,
        linewidth=2,
        hue_norm=(1, 48),
        teeplot_postprocess=f"""
teed.set(xscale="log", ylim=(-0.02, 1.02))
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.02)
teed.set_titles(col_template='{{col_var}} = {{col_name:.0f}}')
""",
    )


## Size Fixation Heatmap


In [ ]:
tp.tee(
    size_fixation_heatmap,
    fixprobs_df=dfxs,
    index="available beneficial mutations",
    on="population size",
    values="fixation probability",
)


## Quality Check: Number Replicates Available


In [ ]:
categories = (
    pl.DataFrame(dfxs)
    .group_by(
        "population size",
    )
    .first()["population size"]
    .sort()
    .cast(pl.String)
    .to_list()
)
categories


In [ ]:
dfy = pl.DataFrame(dfxs).with_columns(
    pl.col("population size")
    .cast(pl.String)
    .cast(pl.Enum(categories))
    .alias("population size cat"),
)
display(dfy.describe()), display(dfy.head()), display(dfy.tail());


In [ ]:
tp.tee(
    sns.displot,
    data=dfy.to_pandas(),
    x="population size cat",
    hue="genotype",
    col="available beneficial mutations",
    col_wrap=3,
    hue_order=["normomutator", "hypermutator"],
    aspect=1.5,
    height=2,
    kind="hist",
    multiple="dodge",
    teeplot_subdir=teeplot_subdir,
    teeplot_outexclude="post",
    teeplot_postprocess=f"""
teed.set(yscale='log')
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.02)
teed.set_titles(col_template='{{col_var}} = {{col_name:.0f}}')
for ax in teed.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_ha("right")
sns.move_legend(
    teed, "lower center",
    bbox_to_anchor=(0.25, 0.97),
    frameon=False,
    ncol=2,
    title=None,
)
""",
)


## Quality Check: Has Reached Fixation?


In [ ]:
dfxs_ = dfxs[dfxs["genotype"] == "hypermutator"].copy().reset_index(drop=True)


In [ ]:
dfxs_["unfixed"] = (dfxs_["fixation probability"] != 1.0) & (
    dfxs_["fixation probability"] > 0.001
)


In [ ]:
tp.tee(
    sns.relplot,
    data=dfxs_[dfxs_["unfixed"]],
    x="population size",
    y="fixation probability",
    hue="genotype",
    col="available beneficial mutations",
    style="genotype",
    col_wrap=3,
    hue_order=["normomutator", "hypermutator"],
    style_order=["normomutator", "hypermutator"],
    aspect=1.5,
    height=2,
    kind="scatter",
    markers=True,
    teeplot_outexclude="post",
    teeplot_subdir=teeplot_subdir,
    teeplot_postprocess=f"""
teed.set(xscale='log')
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.02)
teed.set_titles(col_template='{{col_var}} = {{col_name:.0f}}')
sns.move_legend(
teed,
"lower center",
bbox_to_anchor=(0.25, 0.97),
frameon=False,
ncol=2,
title=None,
)
""",
)


In [ ]:
tp.tee(
    sns.displot,
    data=dfxs_.reset_index(drop=True),
    x="population size",
    hue="unfixed",
    col="available beneficial mutations",
    col_wrap=3,
    aspect=1.5,
    height=2,
    multiple="fill",
    bins=dfxs_["population size"].nunique(),
    log_scale=(True, False),
    teeplot_outexclude="post",
    teeplot_subdir=teeplot_subdir,
    teeplot_postprocess=f"""
teed.figure.suptitle("n={apn} agents per node", x=0.63, y=1.02)
teed.set_titles(col_template='{{col_var}} = {{col_name:.0f}}')
sns.move_legend(
teed,
"lower center",
bbox_to_anchor=(0.25, 0.97),
frameon=False,
ncol=2,
title=None,
)
""",
)
